In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col, to_date, hour
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [ ]:
spark = SparkSession.builder.appName("LogApp").getOrCreate()

In [ ]:
spark.conf.set("spark.sql.legacy.parquet.nanosAsLong", "true")

In [ ]:
schema = StructType([
    StructField("datetime", StringType(), True),
    StructField("ip", StringType(), True),
    StructField("url", StringType(), True),
    StructField("status_code", IntegerType(), True),
    StructField("response_time", IntegerType(), True),
    StructField("user_agent", StringType(), True)
    ])

In [ ]:
df = spark.read.parquet('./logs.parquet', schema=schema)

In [ ]:
df = df.withColumn('timestamp', to_timestamp(col('datetime'), 'yyyy-MM-dd HH:mm:ss'))

In [ ]:
df = df.withColumn("date", to_date(col("timestamp")))
df = df.withColumn("hour", hour(col("timestamp")))

In [ ]:
df.show(10, False)

+-------------------+-------------+----------+-----------+-------------+----------+-------------------+----------+----+
|datetime           |ip           |url       |status_code|response_time|user_agent|timestamp          |date      |hour|
+-------------------+-------------+----------+-----------+-------------+----------+-------------------+----------+----+
|2025-03-15 05:24:45|192.168.0.144|/profile  |500        |73           |Safari    |2025-03-15 05:24:45|2025-03-15|5   |
|2025-03-14 14:06:30|192.168.0.38 |/home     |200        |357          |Firefox   |2025-03-14 14:06:30|2025-03-14|14  |
|2025-03-15 08:29:13|192.168.0.241|/home     |502        |725          |Opera     |2025-03-15 08:29:13|2025-03-15|8   |
|2025-03-14 15:29:50|192.168.0.246|/dashboard|400        |900          |Chrome    |2025-03-14 15:29:50|2025-03-14|15  |
|2025-03-14 10:09:50|192.168.0.100|/login    |201        |552          |Edge      |2025-03-14 10:09:50|2025-03-14|10  |
|2025-03-15 08:25:41|192.168.0.180|/sear

In [ ]:
# cantidad de requests por hora y dia
df.groupBy("date", "hour").count().orderBy("date", "hour").show()

+----------+----+-----+
|      date|hour|count|
+----------+----+-----+
|2025-03-14|  10|41650|
|2025-03-14|  11|41900|
|2025-03-14|  12|41911|
|2025-03-14|  13|41734|
|2025-03-14|  14|41605|
|2025-03-14|  15|41513|
|2025-03-14|  16|41540|
|2025-03-14|  17|41571|
|2025-03-14|  18|41537|
|2025-03-14|  19|41446|
|2025-03-14|  20|41761|
|2025-03-14|  21|41772|
|2025-03-14|  22|41399|
|2025-03-14|  23|41795|
|2025-03-15|   0|41513|
|2025-03-15|   1|41549|
|2025-03-15|   2|41256|
|2025-03-15|   3|41956|
|2025-03-15|   4|41884|
|2025-03-15|   5|41305|
+----------+----+-----+
only showing top 20 rows



In [ ]:
# Top 5 URLS mas visitadas
df.groupBy("url").count().orderBy("count", ascending=False).show(5, False)

+----------+------+
|url       |count |
+----------+------+
|/dashboard|200316|
|/login    |200068|
|/search   |199967|
|/profile  |199835|
|/home     |199814|
+----------+------+



In [ ]:
# Cantidad de errores 4xx y 5xx
df.filter((col("status_code") >= 400) & (col("status_code") < 600)).groupBy("status_code").count().show()

+-----------+------+
|status_code| count|
+-----------+------+
|        502|166536|
|        404|167061|
|        500|166598|
|        400|166264|
+-----------+------+



In [ ]:
# Tiempo de respuesta promedio por URL
df.groupBy("url").avg("response_time").orderBy(col("avg(response_time)").desc()).show()

+----------+------------------+
|       url|avg(response_time)|
+----------+------------------+
|   /search| 505.7301154690524|
|  /profile| 505.6279730777892|
|     /home| 505.0330557418399|
|/dashboard| 504.6675253100102|
|    /login|504.31182397984685|
+----------+------------------+



In [ ]:
# Optimizar DataFrame usando Repartition
optimized_df = spark.read.parquet('logs.parquet').repartition(10)

In [ ]:
# Crear Temporal View
optimized_df.createOrReplaceTempView("logs")

In [ ]:
# Usando Spark SQL para obtener las vistas de las urls.
spark.sql("SELECT url, COUNT(*) as visits FROM logs GROUP BY url ORDER BY visits DESC LIMIT 5").show()

+----------+------+
|       url|visits|
+----------+------+
|/dashboard|200316|
|    /login|200068|
|   /search|199967|
|  /profile|199835|
|     /home|199814|
+----------+------+

